In [50]:
import itertools
import json
import logging
import os
import numpy as np
import pandas as pd

from collections import Counter
from IPython.display import display

pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

# Visu utils - need to make a utils.py file

In [56]:
def color_map(val):
        color_dict = {
            "1": "salmon",
            "<": "grey",
            ">": "grey",
            "v": "grey",
            "^": "grey",
            "P": "pink",
            "W": "blue",
            "D": "purple",
            "C": "grey",
            "_": "brown",
            "O": "green",
            "Q": "red",
            "S": "orange",
            "B": "cyan",
            "R": "lime",
            "F": "yellow",
            "L": "lightblue",
            "0": None
        }
        return 'background-color: %s' % color_dict[val]

def visualize_room(array):
    df_visu = pd.DataFrame(array)
    df_visu = df_visu.style.applymap(color_map)
    return df_visu

def visualize_room_df(df):
    df_visu = df.copy()
    df_visu = df_visu.style.applymap(color_map)
    return df_visu

def visualize_rooms_folder(folder_path):
    cont = input(
        f"There are {len(os.listdir(folder_path))} files in the folder. Do you want to continue? (y/n)\n"
    )
    if cont == "y":
        for fn in os.listdir(folder_path):
            room = pd.read_csv(os.path.join(folder_path, fn), sep=";", header=None, dtype=str)
            display(visualize_room_df(room))


# MdMC utils

In [8]:
class MCModel:
    def __init__(self) -> None:
        self.dmatrix = np.array(
            [
                [0, 0, 0],
                [0, 1, 1],
                [0, 1, 2],
            ]
        )


In [9]:
def submatrix_to_count(array):
    return "".join((array[1, 1], array[1, 2], array[2, 1])), array[2, 2]

def get_all_submatrices(array, xmax=23, ymax=40, size=2):
    all_submatrices = []
    for x in range(xmax-size):
        for y in range(ymax-size):
            all_submatrices.append(array[x:x+3, y:y+3])
    return all_submatrices

## Prepro with only fg

In [10]:
with open("../data_pcg_ready/40x23_fg/lvls_fg.json", "r") as f:
    d_levels = json.load(f)

all_tiles = []
for lv in d_levels.values():
    all_tiles.extend(np.array(lv).flatten())
all_tiles = set(all_tiles)

d_absolute_counts = {}
for perm in itertools.product(all_tiles, repeat=3):
    d_absolute_counts["".join(perm)] = []

for lvl in d_levels.values():
    for submatrix in get_all_submatrices(np.array(lvl)):
        pattern, tiletype = submatrix_to_count(submatrix)
        d_absolute_counts[pattern].append(tiletype)

for key in d_absolute_counts.keys():
    d_absolute_counts[key] = dict(Counter(d_absolute_counts[key]))

d_proba_estimation = {}

for key in d_absolute_counts.keys():
    d_temp = d_absolute_counts[key]
    if d_temp:        
        d_proba_estimation[key] = {k: (v / total) for total in (sum(d_temp.values()),) for k, v in d_temp.items()}

## Pre-processing with recombined maps

In [11]:
all_tiles_and_entities = ["0", "1", "^", "<", "v", ">", "D", "C", "_", "O", "Q", "W", "S", "B", "R", "F", "L"]

In [12]:
data_path = "../data_pcg_ready/rooms_40x23/"
os.listdir(data_path)

['3_lvl_08-d_fg.csv',
 '3_lvl_11-z_fg.csv',
 '7_lvl_d-05b_fg.csv',
 '3_lvl_01-b_fg.csv',
 '6_lvl_boss-15_fg.csv',
 'LostLevels_lvl_e-03-dummy_fg.csv',
 '8_lvl_inside_fg.csv',
 '2_lvl_d6_fg.csv',
 '3_lvl_11-y_fg.csv',
 '5_lvl_b-03_fg.csv',
 '3_lvl_04-c_fg.csv',
 '2_lvl_6_fg.csv',
 '6_lvl_boss-04_fg.csv',
 '3_lvl_12-y_fg.csv',
 '9H_lvl_b-00_fg.csv',
 'LostLevels_lvl_h-04b_fg.csv',
 '1H_lvl_08_fg.csv',
 '4_lvl_a-10_fg.csv',
 '5_lvl_c-10_fg.csv',
 '3X_lvl_02-dummy_fg.csv',
 '1_lvl_3_fg.csv',
 '2_lvl_d7_fg.csv',
 '2_lvl_end_3c_fg.csv',
 '5_lvl_a-10_fg.csv',
 '3_lvl_10-c_fg.csv',
 '6_lvl_boss-05_fg.csv',
 '1_lvl_6c_fg.csv',
 '7_lvl_a-04b_fg.csv',
 '3_lvl_13-x_fg.csv',
 '3_lvl_08-b_fg.csv',
 '6_lvl_b-00c_fg.csv',
 '5_lvl_a-07_fg.csv',
 '3_lvl_08-a_fg.csv',
 '1_lvl_8_fg.csv',
 '5_lvl_a-13_fg.csv',
 '1H_lvl_00_fg.csv',
 '7_lvl_f-02_fg.csv',
 '3_lvl_04-b_fg.csv',
 '3_lvl_07-a_fg.csv',
 '2_lvl_5_fg.csv',
 '8_lvl_outside_fg.csv',
 '5_lvl_a-15_fg.csv',
 '3_lvl_02-a_fg.csv',
 '3_lvl_0x-a_fg.csv',
 '

In [13]:
d_absolute_counts = {}
for perm in itertools.product(all_tiles_and_entities, repeat=3):
    d_absolute_counts["".join(perm)] = []

In [14]:
for fn in os.listdir(data_path):
    lvl_temp = pd.read_csv(os.path.join(data_path, fn), header=None, sep=";", dtype=str).to_numpy(dtype=str)
    for submatrix in get_all_submatrices(np.array(lvl_temp)):
        pattern, tiletype = submatrix_to_count(submatrix)
        d_absolute_counts[pattern].append(tiletype)

In [15]:
for key in d_absolute_counts.keys():
    d_absolute_counts[key] = dict(Counter(d_absolute_counts[key]))

d_proba_estimation = {}

for key in d_absolute_counts.keys():
    d_temp = d_absolute_counts[key]
    if d_temp:
        d_proba_estimation[key] = {k: (v / total) for total in (sum(d_temp.values()),) for k, v in d_temp.items()}

In [16]:
d_proba_estimation

{'000': {'0': 0.9770323060705649,
  'R': 0.0007068890680225708,
  'S': 0.01505549699262107,
  '1': 0.004501767222670056,
  '_': 0.0009177156321696534,
  '^': 0.0009425187573634278,
  'C': 0.00029763750232529297,
  'Q': 0.0003348421901159546,
  '<': 0.00014881875116264649,
  'D': 3.720468779066162e-05,
  'O': 2.4803125193774414e-05},
 '001': {'0': 0.10569327158812312,
  '1': 0.8812312721329338,
  'S': 0.009261781530918006,
  'W': 0.00027240533914464724,
  '>': 0.0005448106782892945,
  '^': 0.002451648052301825,
  '_': 0.0005448106782892945},
 '00^': {'^': 0.8327586206896552,
  '0': 0.15517241379310345,
  '1': 0.006896551724137931,
  '_': 0.0034482758620689655,
  'S': 0.0017241379310344827},
 '00<': {'1': 1.0},
 '00>': {'0': 1.0},
 '00D': {'D': 0.972972972972973, '0': 0.02702702702702703},
 '00C': {'C': 0.6229508196721312, '0': 0.3770491803278688},
 '00_': {'_': 0.7335526315789473,
  '0': 0.2565789473684211,
  '^': 0.006578947368421052,
  '1': 0.003289473684210526},
 '00O': {'0': 1.0},
 

In [17]:
for pattern in d_proba_estimation.keys():
    for symbol in all_tiles_and_entities:
        if symbol not in d_proba_estimation[pattern].keys():
            d_proba_estimation[pattern][symbol] = 0.0

In [18]:
d_proba_estimation

{'000': {'0': 0.9770323060705649,
  'R': 0.0007068890680225708,
  'S': 0.01505549699262107,
  '1': 0.004501767222670056,
  '_': 0.0009177156321696534,
  '^': 0.0009425187573634278,
  'C': 0.00029763750232529297,
  'Q': 0.0003348421901159546,
  '<': 0.00014881875116264649,
  'D': 3.720468779066162e-05,
  'O': 2.4803125193774414e-05,
  'v': 0.0,
  '>': 0.0,
  'W': 0.0,
  'B': 0.0,
  'F': 0.0,
  'L': 0.0},
 '001': {'0': 0.10569327158812312,
  '1': 0.8812312721329338,
  'S': 0.009261781530918006,
  'W': 0.00027240533914464724,
  '>': 0.0005448106782892945,
  '^': 0.002451648052301825,
  '_': 0.0005448106782892945,
  '<': 0.0,
  'v': 0.0,
  'D': 0.0,
  'C': 0.0,
  'O': 0.0,
  'Q': 0.0,
  'B': 0.0,
  'R': 0.0,
  'F': 0.0,
  'L': 0.0},
 '00^': {'^': 0.8327586206896552,
  '0': 0.15517241379310345,
  '1': 0.006896551724137931,
  '_': 0.0034482758620689655,
  'S': 0.0017241379310344827,
  '<': 0.0,
  'v': 0.0,
  '>': 0.0,
  'D': 0.0,
  'C': 0.0,
  'O': 0.0,
  'Q': 0.0,
  'W': 0.0,
  'B': 0.0,
  

In [19]:
with open("./probability_estimation.json", "w") as file:
    json.dump(d_proba_estimation, file, indent=4)

## Room generation

In [133]:
def generate_empty_room(width=40, height=23):
    room = np.zeros((height, width), dtype=str)
    room[0, :] = "1"
    room[:, 0] = "1"
    room[-1, :] = "1"
    room[:, -1] = "1"
    room[1, 1:-1] = "0"
    room[1:-1, 1] = "0"

    return room


def extract_pattern(array, x, y):
    return "".join((array[x+1, y+1], array[x+1, y+2], array[x+2, y+1]))


def extract_proba_from_tile(array, x, y, d_pe=d_proba_estimation):
    pattern = extract_pattern(array, x, y)

    try:
        proba_dist = d_pe[pattern]
    except KeyError: # unseen state
        proba_dist = {}

    return proba_dist 


def generate_new_tile(proba_dist, all_symbols=all_tiles_and_entities, exclude=[]):
    if not proba_dist: # unseen state
        new_tile = np.random.choice(all_symbols)
    else:
        if [key for key in proba_dist.keys() if proba_dist[key] > 0] == exclude:
            raise AssertionError("There is no symbol with a strictly positive proba left!")
        new_tile = np.random.choice(a=list(proba_dist.keys()), p=list(proba_dist.values()))
        while new_tile in exclude:
            new_tile = np.random.choice(a=list(proba_dist.keys()), p=list(proba_dist.values()))
    
    return new_tile


def update_room_array(array, x, y, bt_depth=0, verbose=True):
    
    bt_depth_true = bt_depth_true = min(bt_depth, array.shape[1]-y-3)
    # if bt depth > 0, needs to apply backtracking
    # everytime you pick a symbol, add it to excluded - if cannot pick: random pick
    if bt_depth_true:
        excluded_symbols = {i: [] for i in range(bt_depth_true)}
        k = 0
        array_temp = array.copy()

        while k >= 0 and k < bt_depth_true:
            try:
                pb_dist = extract_proba_from_tile(array_temp, x, y+k)

                if pb_dist:
                    nt = generate_new_tile(pb_dist, exclude=excluded_symbols[k])
                    excluded_symbols[k].append(nt)
                    array_temp[x+2, y+2+k] = nt
                    k += 1
                else: # if pb_dist empty: unseen state
                    k -= 1

            except AssertionError: # no tiles left for level k - need to try something else at k-1
                k -= 1 # backtracking
        
        if k == -1: # no matter the tile used in base lvl it fails => random gen
            logger.warning(
                f"Backtracking failed! Random generation."
            )
            array[x+2, y+2] = generate_new_tile({})
        else: # we made it through: tile at the top level works fine
            array[x+2, y+2] = array_temp[x+2, y+2]

    else:
        array[x+2, y+2] = generate_new_tile(extract_proba_from_tile(array, x, y))
    
    if verbose:
        logger.info(
            f"New tile generated in position ({x+2}, {y+2}): {array[x+2, y+2]}"
        )

    return array


def generate_room(width=40, height=23, backtracking_depth=0, verbose=False):
    room = generate_empty_room(width, height)
    
    for x in range(height-2):
        for y in range(width-2):
            room = update_room_array(room, x, y, bt_depth=backtracking_depth, verbose=verbose)
    
    return room


def generate_room_batch(n_rooms, path_folder_to_save, width=40, height=23, backtracking_depth=0, verbose=False, visualize=False):
    if not os.path.exists(path_folder_to_save):
        os.mkdir(path_folder_to_save)
    for i in range(n_rooms):
        room_temp = generate_room(width, height, backtracking_depth, verbose)
        pd.DataFrame(room_temp).to_csv(f"{path_folder_to_save}/room_{i}_generated_MdMC.csv", header=None, index=None, sep=";")
        logger.info(
            f"Successfully generated room {i}: saved to {path_folder_to_save}/room_{i}_generated_MdMC.csv"
        )
        if visualize:
            display(visualize_room(room_temp))


In [127]:
test_room = generate_room(backtracking_depth=5)
visualize_room(test_room)

INFO:__main__:New tile generated in position (2, 2): 0
INFO:__main__:New tile generated in position (2, 3): 0
INFO:__main__:New tile generated in position (2, 4): 0
INFO:__main__:New tile generated in position (2, 5): 0
INFO:__main__:New tile generated in position (2, 6): 0
INFO:__main__:New tile generated in position (2, 7): 0
INFO:__main__:New tile generated in position (2, 8): 0
INFO:__main__:New tile generated in position (2, 9): 0
INFO:__main__:New tile generated in position (2, 10): 0
INFO:__main__:New tile generated in position (2, 11): 0
INFO:__main__:New tile generated in position (2, 12): 0
INFO:__main__:New tile generated in position (2, 13): 0
INFO:__main__:New tile generated in position (2, 14): 0
INFO:__main__:New tile generated in position (2, 15): 0
INFO:__main__:New tile generated in position (2, 16): 0
INFO:__main__:New tile generated in position (2, 17): 0
INFO:__main__:New tile generated in position (2, 18): 0
INFO:__main__:New tile generated in position (2, 19): 0


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,S,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,R,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,1,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,<
5,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,S,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,1,0,0,0,0,0,S,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,S,0,0,0,0,0,0,0,0,0,0
9,1,0,0,0,0,S,0,0,0,<,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [135]:
generate_room_batch(50, "./generated_maps/test_generation_mdmc/with_bt", width=40, height=23, backtracking_depth=5)

INFO:__main__:Successfully generated room 0: saved to ./generated_maps/test_generation_mdmc/with_bt/room_0_generated_MdMC.csv
INFO:__main__:Successfully generated room 1: saved to ./generated_maps/test_generation_mdmc/with_bt/room_1_generated_MdMC.csv
INFO:__main__:Successfully generated room 2: saved to ./generated_maps/test_generation_mdmc/with_bt/room_2_generated_MdMC.csv
INFO:__main__:Successfully generated room 3: saved to ./generated_maps/test_generation_mdmc/with_bt/room_3_generated_MdMC.csv
INFO:__main__:Successfully generated room 4: saved to ./generated_maps/test_generation_mdmc/with_bt/room_4_generated_MdMC.csv
INFO:__main__:Successfully generated room 5: saved to ./generated_maps/test_generation_mdmc/with_bt/room_5_generated_MdMC.csv
INFO:__main__:Successfully generated room 6: saved to ./generated_maps/test_generation_mdmc/with_bt/room_6_generated_MdMC.csv
INFO:__main__:Successfully generated room 7: saved to ./generated_maps/test_generation_mdmc/with_bt/room_7_generated_M